In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pp
import matplotlib as mpl
import networkx as nx
from networkx.algorithms import bipartite


%matplotlib widget

# Task for Data Analytics Summer Position

In order to create our short list of people for interviews, we would like you to complete a short data analytics problem.  


Based on the information provided at the website below, please complete the questions below and submit them by end of day Thursday, May 2nd.  Please email Dr. Alim if you have any questions about this task.


https://data.calgary.ca/Environment/Corporate-Energy-Consumption/crbp-innf/about_data

 
From the above data, produce a bipartite graph visualization that shows:

* Energy type (Energy Description) in one set of nodes
* Business entity (Business Unit Desc) in the other set of nodes
* Weighted edges that show total amount of each type of energy consumed by each of the business entities (aggregated over the year 2023)
* Include a short written description of the data analysis <250 words.

## Import data and filter to year 2023

In [ ]:
all_data = pd.read_csv('Corporate_Energy_Consumption_20240501.csv', low_memory=False)
data_2023 = all_data[all_data['Year'] == 2023]

print('Columns:')
print(' | '.join(data_2023.columns))

## Define Nodes and Edges from Total Consumption (!! PAY ATTENTION TO UNITS !!)

In [ ]:
kwh2gj = 3.6 * 10**(-3)

In [ ]:
relevant_data = data_2023[['Business Unit Desc', 'Energy Description']] * True

energies = data_2023['Total Consumption']
energies_converted = np.where(data_2023['Unit'] == 'Kwh', energies, energies / kwh2gj)

relevant_data['Total Energy (Kwh)'] = energies_converted

#display(relevant_data[data_2023['Unit'] == 'GJ'])
#display(data_2023[data_2023['Unit'] == 'GJ'])

In [ ]:
relevant_df = relevant_data.groupby(['Business Unit Desc', 'Energy Description'], as_index=False, sort=False).sum()
edges = list(relevant_df.itertuples(index=False,name=None)) 
energy_totals = relevant_df.groupby(
    'Energy Description', as_index=False
).sum().sort_values(by='Total Energy (Kwh)')
business_totals = relevant_df.groupby(
    'Business Unit Desc', as_index=False
).sum().sort_values(by='Total Energy (Kwh)')

## Make Graph

In [ ]:
energy_types = energy_totals['Energy Description'].values
business_units = business_totals['Business Unit Desc'].values

print('Energy Types: ', ' | '.join(energy_types))
print('Business Units: ', ' | '.join(business_units))
print(set(data_2023['Unit']))

In [ ]:
B = nx.Graph()
B.add_nodes_from(energy_types, bipartite=0)
B.add_nodes_from(business_units, bipartite=1)

B.add_weighted_edges_from(edges)

energy_nodes = B.subgraph(energy_types)
business_nodes = B.subgraph(business_units)

## Node sizes based on energy consumption/delivery

In [ ]:
def normalizer(from_arr, to_vmin, to_vmax):
    from_vmin = min(from_arr)
    from_vmax = max(from_arr)
    
    def normalize(x):
        m = (to_vmax - to_vmin) / (from_vmax - from_vmin)
        c = to_vmax - m * from_vmax
        
        return m * x + c
    return normalize

### Energy source node size

In [ ]:


normalize = normalizer([0, energy_totals['Total Energy (Kwh)'].values.max()], 0, 1600)
energy_sizes_dict = dict(
    (source, energy) for i, source, energy in energy_totals.itertuples()
)
energy_size_areas = [normalize(energy_sizes_dict[key]) for key in energy_nodes]



### Business node size

In [ ]:


normalize = normalizer([0, business_totals['Total Energy (Kwh)'].values.max()], 0, 800)
business_sizes_dict = dict(
    (source, energy) for i, source, energy in business_totals.itertuples()
)
business_size_areas = [normalize(business_sizes_dict[key]) for key in business_nodes]



## Defining positions of nodes for visualization

In [ ]:

height = 50
y_energy = np.linspace(0, height, len(energy_types))
y_business = np.linspace(0, height, len(business_units))

pos_left = dict(
    (node, np.array( (0, y_energy[i]) )) for i, node in enumerate(energy_types)
)
pos_right = dict(
    (node, np.array((1, y_business[i]))) for i, node in enumerate(business_units)
)




pos=dict()
pos.update(
    pos_left
)
pos.update(
    pos_right
)


## Visualizing the bipartite graph

In [ ]:
fig = pp.figure()
gs = mpl.gridspec.GridSpec(
    1, 2, width_ratios=[20, 1], wspace=0.01, left= 0.01, right=0.93, top=0.99, bottom=0.01
)

ax = fig.add_subplot(gs[0])
cax = fig.add_subplot(gs[1])


cmap = mpl.cm.viridis_r

fig.set_size_inches((8.5, 6.5))
edge_labels = {
    (node1, node2) : weight['weight'] for node1, node2, weight in B.edges(data=True)
}



label_pos_left = dict(
    (label, xy - [0.1, 0]) for label, xy in pos_left.items()
)
label_pos_right = dict(
    (label, xy + [0.1, 0]) for label, xy in pos_right.items()
)

# Nodes first
color='red'
nx.draw_networkx_nodes(
    energy_nodes, pos=pos, ax=ax, node_size=energy_size_areas, node_color=color, edgecolors='k')
nx.draw_networkx_nodes(business_nodes, pos=pos, ax=ax, node_size=business_size_areas, node_color=color, edgecolors='k')
nx.draw_networkx_labels(energy_nodes, pos=label_pos_left, horizontalalignment='right', ax=ax)
nx.draw_networkx_labels(business_nodes, pos=label_pos_right, horizontalalignment='left', ax=ax)

# Edges

edge_values = [dat['weight'] for _,_, dat in B.edges(data=True)]
edge_weights = np.log10(edge_values)

nx.draw_networkx_edges(
    B, pos=pos, edge_color=edge_weights,
    edge_cmap = cmap,
    width=edge_weights/3,
    ax=ax
)

norm = mpl.colors.LogNorm(
    vmin=min(edge_values), vmax=max(edge_values)
)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=cax, orientation='vertical', label='Energy (Kwh)')
#_ = nx.draw_networkx_edge_labels(B, ax=ax, pos=pos, edge_labels=edge_labels, font_size=5)

ax.set_xlim(-0.6, 2.0)


In [ ]:
fig.savefig('graph.svg')
fig.savefig('graph.pdf')

## Table summary

In [ ]:
pd.options.display.precision = 3

summaries = []
for totals in (energy_totals, business_totals):
    df = totals * True
    df['Total Energy (Kwh)'] *= 10**(-6)
    df.rename(columns = {'Total Energy (Kwh)':'Total Energy (Gwh)'}, inplace=True)
    df['Percentage'] = 100 * df['Total Energy (Gwh)'] / df['Total Energy (Gwh)'].sum() 
    display(df)
    summaries.append(df)

#display(business_totals)

In [ ]:
display(summaries[0].loc[1:].sum())
display(summaries[1].loc[5:].sum())

In [ ]:
f, a = pp.subplots()
a.plot(np.log10(sorted(edge_values)))

In [ ]:
set(all_data[all_data['Business Unit Desc'] == 'Calgary Housing']['Energy Description'])